In [3]:
# Exchange fluxes above the sill
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os

In [4]:
sd = 4 # Sill depth
#/Volumes/Extreme SSD/MITgcm outputs
#/work/oceans/wbao/MITgcm_results/iceplume/fjord_test
#/Users/weiyangbao/.CMVolumes/Caviness-group/iceplume/
file1 = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/2_Qsg_minhs/state_' + str(format(sd,'03d')) + '.nc')
state1 = file1.isel(T=~file1.get_index("T").duplicated())
grid1 = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/2_Qsg_minhs/grid_' + str(format(sd,'03d')) + '.nc')

day = state1.T.data / 86400

vol = grid1.HFacC * grid1.drF * grid1.rA # Z Y X
areaC = grid1.HFacC * grid1.drF * grid1.dyF # Z Y X
areaW = grid1.HFacW * grid1.drF * grid1.dyG # Z Y Xp1
temp = state1.Temp.data
s = state1.S.where(grid1.HFacC>0)
u = state1.U.where(grid1.HFacW>0)
sW = np.where(grid1.HFacC[-1,40,1:] == 0)[-1][0] # Sill west edge index
sE = np.where(grid1.HFacC[-1,40,1:] == 0)[-1][-1] # Sill east edge index
sC = np.ceil((sW+sE)/2).astype(int) # Sill crest index (178)

print('Sill Index: west=' + str(sW) + ',center='+str(sC)+',east='+str(sE))

Sill Index: west=120,center=178,east=235


In [ ]:
grid1.drF, grid1.dyF, grid1.HFacC

In [5]:
uzx = state1.U.data[90:,:,35:45,:].mean(axis=(0,2))
szx = state1.S.data[90:,:,35:45,:].mean(axis=(0,2))
Qw = u[90:,:,35:45,sW].data.mean(0) * areaW[:,35:45,sW].data
Qe = u[90:,:,35:45,sE].data.mean(0) * areaW[:,35:45,sE].data
Fw = s.data[90:,:,35:45,sE].mean(0) * Qw
Fe = s.data[90:,:,35:45,sW].mean(0) * Qe
#Qw[Qw>0].sum()

In [6]:
usE = uzx[:,sE]
usW=uzx[:,sW]
# Layer interface
lW = np.where(usW > 0)[0][0]
lE = np.where(usE > 0)[0][0]
QW = Qw[:lW,:].sum()
QE = Qe[:lE,:].sum()
SW = Fw[:lW,:].sum() / QW
SE = Fe[:lE,:].sum() / QE
#print('Volume flux: west=' + str(QW) + ',east='+str(QE)+'; Salinity: west='+str(SW)+',east='+str(SE))

/home/1959/local/bin/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
/home/1959/local/bin/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [9]:
szx[:,sE] - szx[:,sW]

array([  0.04838562,   0.15340233,   0.33768272,   0.58280563,
         0.9275856 ,   1.4185638 ,   1.855154  ,   2.0773735 ,
         2.1535702 ,   2.187025  ,   2.2113495 ,   2.2323112 ,
         2.2519417 ,   2.269617  ,   2.2842445 ,   2.2961864 ,
         2.3067818 ,   2.3171616 ,   2.3277798 ,   2.3387547 ,
         2.3541145 ,   2.3819847 ,   2.420658  ,   2.4645252 ,
         2.5096416 ,   2.5526886 ,   2.591999  ,   2.627493  ,
         2.6598644 ,   2.6897316 ,   2.7173443 ,   2.7430344 ,
         2.7675323 ,   2.790985  ,   2.8138504 ,   2.8360977 ,
         2.8576946 ,   2.8782578 ,   2.8978767 ,   2.9164562 ,
         2.9340115 ,   2.950325  ,   2.965269  ,   2.9787197 ,
         2.9903545 ,   3.0003796 ,   3.0088882 ,   3.015833  ,
         3.021534  ,   3.0262241 ,   3.030037  ,   3.0323315 ,
         3.033331  ,   3.0330334 ,   3.0319824 ,   3.030201  ,
         3.027502  ,   3.0237617 ,   3.0192223 ,   3.0137043 ,
         3.0070248 ,   2.9994564 ,   2.99049   ,   2.98

In [ ]:
Qw[Qw>0].sum(), Qe[Qe>0].sum(), Qw[Qw<0].sum(), Qe[Qe<0].sum()

In [ ]:

kk = np.where(usE > 0)
np.argwhere(np.diff(kk)>2)[0][-1]

In [ ]:
# Exclude the upper layer for the TEF calculation
S = state1.S.data
U = (state1.U.data[:,:,:,1:] + state1.U.data[:,:,:,:-1]) / 2
Uzx = U[89:,:,35:45,:].mean(axis=(0,2))
for j in range(Uzx.shape[1]):
    if any(Uzx[:,j]>0)==False:
        pass
    else:
        mid = np.where(Uzx[:,j]>0)[0][0]
        if mid>0:
            S[:,:mid,:,j] = 0
            U[:,:mid,:,j] = 0
        else:
            pass

In [ ]:
U[89:,:,35:45,235].mean(axis=(0,2)), S[:,:,35:45,235].mean(axis=(0,2))

In [ ]:
S = state1.S.data
U = (state1.U.data[:,:,:,1:] + state1.U.data[:,:,:,:-1]) / 2
Uzx = U[89:,:,35:45,:].mean(axis=(0,2))
up = np.zeros(Uzx.shape[1])
mi = np.zeros(Uzx.shape[1])
for j in range(Uzx.shape[1]):
    uz = Uzx[:,j]
    sign = np.where(np.sign(uz[:-1]) != np.sign(uz[1:]))[0] + 1
    if sign.size==0:
        pass
    elif sign.size==1:
        mi[j] = sign[0]
    else:
        up[j] = sign[0]
        mi[j] = sign[1]
    
    if np.abs(up[j]-mi[j])<5:
        mi[j] = up[j]
        up[j] = 0

In [ ]:
a = Uzx[:,1]
s = np.where(np.sign(a[:-1]) != np.sign(a[1:]))[0] + 1

In [ ]:
mid = np.where(Uzx[:,43]>0)[0][0]
low = np.where(Uzx[:,43] < 0)[-1][0]
low

In [ ]:
up, mi

In [ ]:
up,  mi